# OpenFermion-Cirq hands-on tutorial

OpenFermion is a library for obtaining Hamiltonian representations and performing pre-processing tasks on them. OpenFermion-Cirq compiles these Hamiltonian representations into quantum simulation circuits in Cirq.

As part of the OpenFermion hands-on tutorial, we demonstrated how to generate a Hubbard model Hamiltonian as a FermionOperator. Here we'll use a spinless model so that the circuits are a more manageable size.

The spinless Hubbard model has the Hamiltonian

$$
H = - t \sum_{\langle i,j \rangle}
             (a^\dagger_{i} a_{j} +
              a^\dagger_{j} a_{i})
     + U \sum_{\langle i, j \rangle} a^\dagger_{i} a_{i}
                  a^\dagger_{j} a_{j}
$$

where $\langle i, j \rangle$ ranges over the edges in a 2-dimensional grid.

In [1]:
import openfermion as of

# Set Hubbard model parameters
x_dim = 2
y_dim = 3
tunneling = 1.0
coulomb = 4.0
spinless=True

# Create Hubbard model Hamiltonian as FermionOperator
hubbard_model = of.fermi_hubbard(x_dim, y_dim, tunneling, coulomb, spinless=spinless)

print(hubbard_model)

4.0 [0^ 0 1^ 1] +
4.0 [0^ 0 2^ 2] +
-1.0 [0^ 1] +
-1.0 [0^ 2] +
-1.0 [0^ 4] +
-1.0 [1^ 0] +
4.0 [1^ 1 3^ 3] +
-1.0 [1^ 3] +
-1.0 [1^ 5] +
-1.0 [2^ 0] +
4.0 [2^ 2 3^ 3] +
4.0 [2^ 2 4^ 4] +
-1.0 [2^ 3] +
-1.0 [2^ 4] +
-1.0 [3^ 1] +
-1.0 [3^ 2] +
4.0 [3^ 3 5^ 5] +
-1.0 [3^ 5] +
-1.0 [4^ 0] +
-1.0 [4^ 2] +
4.0 [4^ 4 0^ 0] +
4.0 [4^ 4 5^ 5] +
-1.0 [4^ 5] +
-1.0 [5^ 1] +
-1.0 [5^ 3] +
-1.0 [5^ 4] +
4.0 [5^ 5 1^ 1]


In this tutorial we'll demonstrate how to obtain circuits to simulate time evolution by this Hamiltonian and also use a variational algorithm to approximate its ground energy.

## Hamiltonian simulation via Trotter formulas

- Goal: apply $\exp(-i H t)$ where $H = \sum_j H_j$
- Use an approximation such as $\exp(-i H t) \approx (\prod_{j=1} \exp(-i H_j t/r))^r$
- Exposed via the `simulate_trotter` function
- Currently implemented algorithms are from [arXiv:1706.00023](https://arxiv.org/pdf/1706.00023.pdf), [arXiv:1711.04789](https://arxiv.org/pdf/1711.04789.pdf), and [arXiv:1808.02625](https://arxiv.org/pdf/1808.02625.pdf), and are based on the JWT
- Currently supported Hamiltonian types: DiagonalCoulombHamiltonian and InteractionOperator

As a demonstration, we'll simulate time evolution under the Hubbard model Hamiltonian we generated earlier.

First, let's create a random initial state and apply the exact time evolution by matrix exponentiation:
$$
\lvert \psi \rangle \mapsto \exp(-i H t) \lvert \psi \rangle
$$

In [2]:
from scipy.sparse.linalg import expm_multiply

# Create a random initial state
n_qubits = of.count_qubits(hubbard_model)
initial_state = of.haar_random_vector(2**n_qubits, seed=7)

# Convert Hamiltonian to sparse matrix
hubbard_sparse = of.get_sparse_operator(of.jordan_wigner(hubbard_model))

# Set evolution time
time = 1.0

# Apply exp(-i H t) to the state
exact_state = expm_multiply(-1j*hubbard_sparse*time, initial_state)

Now, let's create a circuit to perform the evolution and compare the fidelity of the resulting state with the one from exact evolution. The fidelity can be increased by increasing the number of Trotter steps or the order of the formula. Note that the Hamiltonian input to `simulate_trotter` should be a DiagonalCoulombHamiltonian, not a FermionOperator.

In [3]:
import cirq
import openfermioncirq as ofc
import numpy as np

# Convert Hamiltonian to a DiagonalCoulombHamiltonian instance
hubbard_hamiltonian = of.get_diagonal_coulomb_hamiltonian(hubbard_model)

# Initialize qubits
qubits = cirq.LineQubit.range(n_qubits)

# Create circuit
circuit = cirq.Circuit.from_ops(
    ofc.simulate_trotter(
        qubits, hubbard_hamiltonian, time,
        n_steps=10,
        order=0,
        algorithm=ofc.trotter.LINEAR_SWAP_NETWORK)
)

# Apply the circuit to the initial state
result = circuit.apply_unitary_effect_to_state(initial_state)

# Compute the fidelity with the final state from exact evolution
fidelity = abs(np.dot(exact_state, result.conj()))**2

print(fidelity)

0.9732473269809816


### Estimating hardware resources

The circuit we generated above is expressed in terms of gates convenient for the algorithm, not for a specific hardware platform. Below, we print out a smaller version of the circuit (with a single Trotter step).

In [4]:
# Create circuit
circuit = cirq.Circuit.from_ops(
    ofc.simulate_trotter(
        qubits, hubbard_hamiltonian, time,
        n_steps=1,
        order=0,
        algorithm=ofc.trotter.LINEAR_SWAP_NETWORK,
        omit_final_swaps=True),
    strategy=cirq.InsertStrategy.EARLIEST
)

cirq.DropNegligible().optimize_circuit(circuit)
print(circuit.to_text_diagram(transpose=True))

0    1           2           3           4           5
│    │           │           │           │           │
XXYY─XXYY^-0.637 XXYY────────XXYY^-0.637 XXYY────────XXYY^-0.637
│    │           │           │           │           │
@────@^0.727     @───────────@^0.727     @───────────@^0.727
│    │           │           │           │           │
×ᶠ───×ᶠ          ×ᶠ──────────×ᶠ          ×ᶠ──────────×ᶠ
│    │           │           │           │           │
│    ×ᶠ──────────×ᶠ          ×ᶠ──────────×ᶠ          │
│    │           │           │           │           │
XXYY─XXYY^-0.637 │           │           XXYY────────XXYY^-0.637
│    │           │           │           │           │
@────@^0.727     │           │           @───────────@^0.727
│    │           │           │           │           │
×ᶠ───×ᶠ          ×ᶠ──────────×ᶠ          ×ᶠ──────────×ᶠ
│    │           │           │           │           │
│    XXYY────────XXYY^-0.637 XXYY────────XXYY^-0.637 │
│    │           │           │ 

Let's convert the circuit to a form suitable for running on Google's Xmon hardware. The following line of code decomposes the gates of our circuit into the Xmon gate set and also performs optimizations such as merging single-qubit rotations.

In [5]:
xmon_circuit = cirq.google.optimized_for_xmon(circuit)

print(xmon_circuit.to_text_diagram(transpose=True))

0               1                2               3               4               5
│               │                │               │               │               │
W(0.549)^-0.5   │                W(0.75)^0.5     │               W(0.75)^0.5     │
│               │                │               │               │               │
@───────────────@                @───────────────@               @───────────────@
│               │                │               │               │               │
W(0.549)^0.5    W(0.0488)^-0.818 W(0.75)^-0.5    W(0.25)^0.818   W(0.75)^-0.5    W(0.25)^0.818
│               │                │               │               │               │
@───────────────@                @───────────────@               @───────────────@
│               │                │               │               │               │
W(0.412)^0.5    W(0.0488)^0.818  W(0.613)^0.5    W(0.25)^-0.818  W(0.613)^0.5    W(0.25)^-0.818
│               │                │               │            

The Xmon gate set contains one type of two-qubit gate, Exp11Gate. These gates will be the limiting resource in the near term. Let's count how many gates of this type appear in our circuit.

In [6]:
exp11gates = xmon_circuit.findall_operations_with_gate_type(cirq.google.Exp11Gate)
print(len(list(exp11gates)))

39


## Variational algorithms (VQE)

- Optimize a parameterized quantum circuit applied to an initial state to minimize a cost function
- Usually, the cost function is
$$
E(\vec \theta) =  \langle \psi \rvert
U^\dagger(\vec{\theta}) H U(\vec{\theta})
\lvert \psi\rangle.
$$
- The parameterized circuit $U(\vec{\theta})$ is called an ansatz
- A popular choice is to use an ansatz of the form
$$
U(\vec{\theta}) = \prod_j \exp(-i \theta_j H_j)
$$
where $H = \sum_j H_j$
- OpenFermion-Cirq contains some built-in ansatzes of this form based on Trotter steps used in Hamiltonian simulation.

In [7]:
ansatz = ofc.SwapNetworkTrotterAnsatz(
    hubbard_hamiltonian,
    iterations=3)
print(ansatz.circuit.to_text_diagram(transpose=True))

0    1            2            3            4            5
│    │            │            │            │            │
XXYY─XXYY^T_0_1_0 XXYY─────────XXYY^T_2_3_0 XXYY─────────XXYY^T_4_5_0
│    │            │            │            │            │
@────@^V_0_1_0    @────────────@^V_2_3_0    @────────────@^V_4_5_0
│    │            │            │            │            │
×ᶠ───×ᶠ           ×ᶠ───────────×ᶠ           ×ᶠ───────────×ᶠ
│    │            │            │            │            │
│    ×ᶠ───────────×ᶠ           ×ᶠ───────────×ᶠ           │
│    │            │            │            │            │
XXYY─XXYY^T_1_3_0 ×ᶠ───────────×ᶠ           XXYY─────────XXYY^T_2_4_0
│    │            │            │            │            │
@────@^V_1_3_0    │            │            @────────────@^V_2_4_0
│    │            │            │            │            │
×ᶠ───×ᶠ           │            │            ×ᶠ───────────×ᶠ
│    │            │            │            │            │
│    XXYY───────

Take another look at the Hubbard model Hamiltonian,
$$
H = - t \sum_{\langle i,j \rangle}
             (a^\dagger_{i} a_{j} +
              a^\dagger_{j} a_{i})
     + U \sum_{\langle i, j \rangle} a^\dagger_{i} a_{i}
                  a^\dagger_{j} a_{j}
$$
and notice that it conserves particle number. That is, each of its terms contains exactly as many creation operators as annihilation operators. This is equivalent to the statement that $H$ commutes with the *total occupation number operator*,
$$
\sum_{j} a_j^\dagger a_j
$$

When studying particle-number-conserving Hamiltonians, one can restrict attention to a particular eigenspace of the total occupation number operator, corresponding to states with a fixed, well-defined number of particles. Time evolution by the Hamiltonian will not bring states out of the eigenspace.

In this example, we'll try to approximate the ground state energy of the Hamiltonian at 2 particles.

In [8]:
n_electrons = 2

# Compute the ground state energy at the specified number of electrons
hamiltonian_sparse = of.get_sparse_operator(hubbard_model)
true_ground_energy, _ = of.jw_get_ground_state_at_particle_number(
    hamiltonian_sparse, n_electrons)

print(true_ground_energy)

-3.1231056256176655


Optimizing an ansatz requires creating a VariationalStudy object, which encapsulates the ansatz, the objective function, and state preparation circuit. Objective functions are represented by subclasses of the VariationalObjective class. Usually, one will simply create an instance of the HamiltonianObjective subclass. A VariationalStudy automatically saves all optimization runs performed and can be saved to and loaded from disk.

Recall that the Hubbard model has the form of a DiagonalCoulombHamiltonian:
$$
\sum_{pq} T_{pq} a_p^\dagger a_q + \sum_{pq} V_{pq} a_p^\dagger a_p a_q^\dagger a_q
$$

The one-body term by itself is a quadratic Hamiltonian; we'll use an eigenstate of it as our reference state. The `prepare_gaussian_state` function can be used to prepare eigenstates of quadratic Hamiltonians.

The built-in ansatzes also feature a starting guess for the initial parameters which is inspired by the idea of state preparation by adiabatic evolution.

In [9]:
# Use preparation circuit for eigenstate of one-body term
preparation_circuit = cirq.Circuit.from_ops(
    ofc.prepare_gaussian_state(
        ansatz.qubits,
        of.QuadraticHamiltonian(hubbard_hamiltonian.one_body),
        occupied_orbitals=range(n_electrons)
    )
)

# Define the objective function
objective = ofc.HamiltonianObjective(hubbard_hamiltonian)

# Create a variational study
study = ofc.VariationalStudy(
    name='hubbard_study',
    ansatz=ansatz,
    objective=objective,
    preparation_circuit=preparation_circuit)

print("Created a variational study with {} parameters.".format(
    study.num_params)
)
print()

print("The energy of the default initial guess is {}.".format(
    study.value_of(ansatz.default_initial_params()))
)
print()

print("The initial guess is:")
print(ansatz.default_initial_params())

Created a variational study with 54 parameters.

The energy of the default initial guess is 1.653323771569472.

The initial guess is:
[ 0.18028137  0.72676046  0.18028137  0.72676046  0.18028137  0.72676046
  0.18028137  0.72676046  0.18028137  0.72676046  0.18028137  0.72676046
  0.18028137  0.72676046  0.18028137  0.72676046  0.18028137  0.72676046
  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137
  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137
  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137
  0.18028137 -0.36619772  0.18028137 -0.36619772  0.18028137 -0.36619772
  0.18028137 -0.36619772  0.18028137 -0.36619772  0.18028137 -0.36619772
  0.18028137 -0.36619772  0.18028137 -0.36619772  0.18028137 -0.36619772]


Performing an optimization run requires creating an OptimizationParams object. The most important part of this is the optimization algorithm to use.

In [10]:
# Perform an optimization run.
algorithm = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'COBYLA'},
    uses_bounds=False)
optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algorithm,
    initial_guess=ansatz.default_initial_params())
result = study.optimize(optimization_params)

print("Optimized energy: {}".format(result.optimal_value))
print()
print("Optimized parameters:")
print(result.optimal_parameters)

Optimized energy: -3.1048166845231027

Optimized parameters:
[ 1.10176866  0.9079445   0.18088205  1.76436972  0.22767709  1.59608786
  0.13556406  0.66440617  0.14740382  0.6565388   0.38062588  0.76870222
  0.25305662  0.48496756  0.18418796  0.85658855  0.08537626  0.70064499
  0.2335307   0.0873193   0.18416921  1.421086    0.08689049  1.1715218
  0.16307193  1.18230601  0.03200061  0.28494296  0.24487017  0.36966998
  0.23818851 -0.171142    0.2479021   0.00337084  0.25920618  0.17142982
  0.27410224 -0.32818709  0.18814917  0.24622833  0.26641112  0.57843257
  0.22577714  0.83967355  0.23824114  0.81714914  0.24715815 -0.40711872
  0.12783937  0.92890347  0.17162613  0.50291806  0.20984109 -0.39022718]


### Creating your own ansatz

To demonstrate how to create and optimize your own ansatz, we will design an ansatz to compute the ground state energy of H2.

In [11]:
import openfermionpyscf as ofpyscf

# Set molecule parameters
geometry = [('H', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 0.7414))]
basis = 'sto-3g'
multiplicity = 1
charge = 0

# Perform electronic structure calculations and
# obtain Hamiltonian as an InteractionOperator
hamiltonian = ofpyscf.generate_molecular_hamiltonian(
    geometry, basis, multiplicity, charge)

# Compute and print ground state energy
ground_energy, _ = of.get_ground_state(
    of.get_sparse_operator(hamiltonian))
print(ground_energy)

-1.137270174660897


We'll use as our reference state the Hartree-Fock state, which in this case is simply
$$
\lvert \psi \rangle = a^\dagger_1 a^\dagger_0 \vert \text{vac} \rangle
$$
Under the JWT, this is equal to
$$
X_1 X_0 \vert 0 0 0 0 \rangle = \vert 1 1 0 0 \rangle
$$
It turns out that the ground state can be reached by applying a single-parameter unitary of the form
$$
\exp (-i \theta Y_3 X_2 X_1 X_0)
$$

First, let's define an ansatz to perform this unitary. We define an ansatz by subclassing VariationalAnsatz.

In [12]:
class MyAnsatz(ofc.VariationalAnsatz):

    def params(self):
        """The parameters of the ansatz."""
        return [cirq.Symbol('theta_0')]

    def operations(self, qubits):
        """Produce the operations of the ansatz circuit."""
        q0, q1, q2, q3 = qubits
        yield cirq.H(q0), cirq.H(q1), cirq.H(q2)
        yield cirq.RotXGate(half_turns=-0.5).on(q3)

        yield cirq.CNOT(q0, q1), cirq.CNOT(q1, q2), cirq.CNOT(q2, q3)
        yield cirq.RotZGate(half_turns=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2, q3), cirq.CNOT(q1, q2), cirq.CNOT(q0, q1)

        yield cirq.H(q0), cirq.H(q1), cirq.H(q2)
        yield cirq.RotXGate(half_turns=0.5).on(q3)

    def _generate_qubits(self):
        """Produce qubits that can be used by the ansatz circuit."""
        return cirq.LineQubit.range(4)

In [13]:
ansatz = MyAnsatz()
print(ansatz.circuit)

0: ───H────────@───────────────────────────────────@───H───
               │                                   │
1: ───H────────X───@───────────────────────@───────X───H───
                   │                       │
2: ───H────────────X───@───────────────@───X───────H───────
                       │               │
3: ───X^-0.5───────────X───Z^theta_0───X───X^0.5───────────


Now let's create a variational study.

In [14]:
# Define the objective function
objective = ofc.HamiltonianObjective(hamiltonian)

# Create preparation circuit
q0, q1, _, _ = ansatz.qubits
preparation_circuit = cirq.Circuit.from_ops(cirq.X(q0), cirq.X(q1))

# Create variational study
study = ofc.VariationalStudy(
    name='my_hydrogen_study',
    ansatz=ansatz,
    objective=objective,
    preparation_circuit=preparation_circuit)

print(study.circuit)

0: ───X───H────────@───────────────────────────────────@───H───
                   │                                   │
1: ───X───H────────X───@───────────────────────@───────X───H───
                       │                       │
2: ───────H────────────X───@───────────────@───X───────H───────
                           │               │
3: ───────X^-0.5───────────X───Z^theta_0───X───X^0.5───────────


Finally, let's optimize the parameters.

In [15]:
# Choose an initial guess
initial_guess = [0.01]
print('Value of initial guess: {}'.format(study.value_of(initial_guess)))

# Choose an algorithm
algorithm = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'L-BFGS-B'},
    options={'maxiter': 100})

# Create OptimizationParams object
optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algorithm,
    initial_guess=initial_guess)

# Run optimization
result = study.optimize(optimization_params)
print('Optimized value: {}'.format(result.optimal_value))

Value of initial guess: -1.1219899918382532
Optimized value: -1.1372701746608995


## Circuit primitives (building blocks)

The algorithms in OpenFermion-Cirq use common circuit primitives which are very useful for constructing low-depth circuits for quantum simulation. In this section we explain these primitives and how to use them.

### Swap network

- Method for applying arbitrary pairwise interactions between qubits or fermionic modes in linear depth using linear connectivity.
- Uses a parallelized network of swap gates to reverse the order of qubits.
- Pairwise operations are applied when the appropriate logical qubits become adjacent.
- Introduced by [arXiv:1711.04789](https://arxiv.org/pdf/1711.04789.pdf)

In [16]:
qubits = cirq.LineQubit.range(5)
circuit = cirq.Circuit.from_ops(ofc.swap_network(qubits))

print(circuit.to_text_diagram(transpose=True))

0 1 2 3 4
│ │ │ │ │
×─× ×─× │
│ │ │ │ │
│ ×─× ×─×
│ │ │ │ │
×─× ×─× │
│ │ │ │ │
│ ×─× ×─×
│ │ │ │ │
×─× ×─× │
│ │ │ │ │



In [17]:
def cz_interaction(i, j, a, b):
    return cirq.Rot11Gate(half_turns=cirq.Symbol('theta_{}{}'.format(i, j))).on(a, b)

circuit = cirq.Circuit.from_ops(
    ofc.swap_network(qubits, cz_interaction),
    strategy=cirq.InsertStrategy.EARLIEST)

print(circuit.to_text_diagram(transpose=True))

0 1          2          3          4
│ │          │          │          │
@─@^theta_01 @──────────@^theta_23 │
│ │          │          │          │
×─×          ×──────────×          │
│ │          │          │          │
│ @──────────@^theta_03 @──────────@^theta_24
│ │          │          │          │
│ ×──────────×          ×──────────×
│ │          │          │          │
@─@^theta_13 @──────────@^theta_04 │
│ │          │          │          │
×─×          ×──────────×          │
│ │          │          │          │
│ @──────────@^theta_14 @──────────@^theta_02
│ │          │          │          │
│ ×──────────×          ×──────────×
│ │          │          │          │
@─@^theta_34 @──────────@^theta_12 │
│ │          │          │          │
×─×          ×──────────×          │
│ │          │          │          │



In [18]:
circuit = cirq.Circuit.from_ops(
    ofc.swap_network(
        qubits,
        lambda i, j, a, b: ofc.XXYY(a, b) if abs(i-j) == 1
                           else cirq.CZ(a, b),
        fermionic=True),
    strategy=cirq.InsertStrategy.EARLIEST)
print(circuit.to_text_diagram(transpose=True))

0    1    2    3    4
│    │    │    │    │
XXYY─XXYY XXYY─XXYY │
│    │    │    │    │
×ᶠ───×ᶠ   ×ᶠ───×ᶠ   │
│    │    │    │    │
│    @────@    @────@
│    │    │    │    │
│    ×ᶠ───×ᶠ   ×ᶠ───×ᶠ
│    │    │    │    │
@────@    @────@    │
│    │    │    │    │
×ᶠ───×ᶠ   ×ᶠ───×ᶠ   │
│    │    │    │    │
│    @────@    @────@
│    │    │    │    │
│    ×ᶠ───×ᶠ   ×ᶠ───×ᶠ
│    │    │    │    │
XXYY─XXYY XXYY─XXYY │
│    │    │    │    │
×ᶠ───×ᶠ   ×ᶠ───×ᶠ   │
│    │    │    │    │



As an example use case, we will use a swap network to simulate the dynamics of a fully-connected Ising model Hamiltonian
$$
H = \sum_{ij} J_{ij} Z_i Z_j
$$

First, let's generate a random Ising model Hamiltonian as a QubitOperator.

In [19]:
import openfermion as of
import itertools
import numpy as np

n_qubits = 5

ising_model = of.QubitOperator()

for i, j in itertools.combinations(range(n_qubits), 2):
    ising_model += of.QubitOperator(((i, 'Z'), (j, 'Z')), np.random.randn())

print(ising_model)

0.16518371027018444 [Z0 Z1] +
0.8991964637134117 [Z0 Z2] +
0.8090460084237492 [Z0 Z3] +
0.09825423160650269 [Z0 Z4] +
-0.888023316687039 [Z1 Z2] +
-0.8253222167707652 [Z1 Z3] +
1.1780366055294307 [Z1 Z4] +
-1.0923635176032285 [Z2 Z3] +
0.41831704437596656 [Z2 Z4] +
0.4572184660768701 [Z3 Z4]


Now let's create a random initial state and apply $\exp(-i H t)$ to it, for $t=1$.

In [20]:
# Create a random initial state
initial_state = of.haar_random_vector(2**n_qubits)

# Convert Hamiltonian to sparse matrix
ising_sparse = of.get_sparse_operator(ising_model)

# Set evolution time
time = 1.0

# Apply exp(-i H t) to the state
final_state = expm_multiply(-1j*ising_sparse*time, initial_state)

If `initial_state` is $\lvert \psi \rangle$, then `final_state` is now $\exp(-i H t) \lvert \psi \rangle$.

### Exercise

Fill in the code cell below to construct a circuit which applies $\exp(-i H t)$ using a swap network. Note the following:
- You'll want to use the `ofc.ZZGate` gate. The most convenient way to initialize it in this case is to use the `duration` argument.
- A swap network reverses the order of qubits. Therefore, to get the circuit to simulate properly, you'll need to swap the qubits back with another swap network. In a real experiment we would just keep track of the qubit order instead of performing these extra swaps.

In [21]:
# Initialize qubits
qubits = cirq.LineQubit.range(n_qubits)

# Write code below to create the circuit
# You should define the `circuit` variable here
# ---------------------------------------------


# ---------------------------------------------

# Apply the circuit to the initial state
result = circuit.apply_unitary_effect_to_state(initial_state)

# Compute the fidelity with the correct final state
fidelity = abs(np.dot(final_state, result.conj()))**2

# Print fidelity; it should be 1
print(fidelity)

0.02916386574845472


### Bogoliubov transformation

- Single-particle orbital basis change
- In the particle-conserving case, takes the form
$$
U a_p^\dagger U^\dagger = b_p^\dagger, \quad b_p^\dagger = \sum_{q} u_{pq} a_q^\dagger
$$
and $u$ is unitary.
- Can be used to diagonalize any quadratic Hamiltonian:
$$
\sum_{p, q} T_{pq} a_p^\dagger a_q \mapsto \sum_{j} \varepsilon_j b_j^\dagger b_j + \text{constant}
$$
- Implementation from [arXiv:1711.05395](https://arxiv.org/pdf/1711.05395.pdf); uses linear depth and linear connectivity

As an example, we'll prepare the ground state of a random particle-conserving quadratic Hamiltonian

In [22]:
n_qubits = 5
quad_ham = of.random_quadratic_hamiltonian(
    n_qubits, conserves_particle_number=True, seed=7)

print(of.get_fermion_operator(quad_ham))

1.690525703800356 [] +
(0.5315776978980016+0j) [0^ 0] +
(-1.347208023348913+2.7004721387490935j) [0^ 1] +
(-0.28362365442898696-1.8784499457335426j) [0^ 2] +
(0.12594647819298657-1.3106154125325498j) [0^ 3] +
(-0.3880303291443195-1.1751249212322041j) [0^ 4] +
(-1.347208023348913-2.7004721387490935j) [1^ 0] +
(2.5012533818678193+0j) [1^ 1] +
(0.3391421007279024-3.8305756810505094j) [1^ 2] +
(-0.3509690502067961+0.090677856754656j) [1^ 3] +
(1.8575239595653907-1.4736314761076197j) [1^ 4] +
(-0.28362365442898696+1.8784499457335426j) [2^ 0] +
(0.3391421007279024+3.8305756810505094j) [2^ 1] +
(-0.019560786804260433+0j) [2^ 2] +
(-2.979765944360631+2.5490724453105917j) [2^ 3] +
(0.5091942820312417+0.344618218148502j) [2^ 4] +
(0.12594647819298657+1.3106154125325498j) [3^ 0] +
(-0.3509690502067961-0.090677856754656j) [3^ 1] +
(-2.979765944360631-2.5490724453105917j) [3^ 2] +
(3.767336752913784+0j) [3^ 3] +
(-1.2963431636902167-1.5288970105744286j) [3^ 4] +
(-0.3880303291443195+1.1751249212322

Now we construct a circuit which maps computational basis states to eigenstates of the Hamiltonian.

In [23]:
circuit = cirq.Circuit.from_ops(
    ofc.bogoliubov_transform(
        qubits,
        quad_ham.diagonalizing_bogoliubov_transform()))

print(circuit.to_text_diagram(transpose=True))

0     1        2        3        4
│     │        │        │        │
Z^0.0 Z^0.0    Z        Z        Z^-3.33e-16
│     │        │        │        │
│     │        │        YXXY─────#2^0.0727
│     │        │        │        │
│     │        YXXY─────#2^0.455 Z^-0.777
│     │        │        │        │
│     YXXY─────#2^0.814 Z^-0.508 │
│     │        │        │        │
│     │        Z^-0.304 YXXY─────#2^0.381
│     │        │        │        │
YXXY──#2^0.703 │        │        Z^0.401
│     │        │        │        │
│     Z^-0.767 YXXY─────#2^0.813 │
│     │        │        │        │
│     YXXY─────#2^0.636 Z^-0.928 │
│     │        │        │        │
│     │        Z^0.393  YXXY─────#2^0.481
│     │        │        │        │
│     │        YXXY─────#2^0.422 Z^-0.738
│     │        │        │        │
│     │        │        Z^-0.824 │
│     │        │        │        │
│     │        │        YXXY─────#2^0.506
│     │        │        │        │
│     │        │        │      

In the rotated basis, the quadratic Hamiltonian takes the form
$$
H = \sum_j \varepsilon_j b_j^\dagger b_j + \text{constant}
$$
We can get the $\varepsilon_j$ and the constant using the `orbital_energies` method of QuadraticHamiltonian.

In [24]:
orbital_energies, constant = quad_ham.orbital_energies()

print(orbital_energies)
print(constant)

[-6.25377614 -1.2291963   0.71202361  5.0062515   8.20078604]
1.690525703800356


The ground state of the Hamiltonian is prepared by filling in the orbitals with negative energy.

In [25]:
# Apply the circuit with initial state having the first two modes occupied.
result = circuit.apply_unitary_effect_to_state(0b11000)

# Compute the expectation value of the final state with the Hamiltonian
quad_ham_sparse = of.get_sparse_operator(quad_ham)
print(of.expectation(quad_ham_sparse, result))

# Print out the ground state energy; it should match
print(quad_ham.ground_energy())

(-5.792446738060052-1.1102230246251565e-16j)
-5.792446738060046


Recall that the Jordan-Wigner transform of $b_j^\dagger b_j$ is $\frac12(I-Z)$. Therefore, $\exp(-i \varepsilon_j b_j^\dagger b_j)$ is equivalent to a single-qubit Z rotation under the JWT. Since the operators $b_j^\dagger b_j$ commute, we have
$$
\exp(-i H t) = \exp(-i \sum_j \varepsilon_j b_j^\dagger b_j t)
= \prod_j \exp(-i \varepsilon_j b_j^\dagger b_j t)
$$
This gives a method for simulating time evolution under a quadratic Hamiltonian:
- Use a Bogoliubov transformation to change to the basis in which the Hamiltonian is diagonal (Note: this transformation might be the inverse of what you expect. In that case, use `cirq.inverse`)
- Apply single-qubit Z-rotations with angles proportional to the orbital energies
- Undo the basis change

The code cell below creates a random initial state and applies time evolution by direct matrix exponentiation.

In [26]:
# Create a random initial state
initial_state = of.haar_random_vector(2**n_qubits)

# Set evolution time
time = 1.0

# Apply exp(-i H t) to the state
final_state = expm_multiply(-1j*quad_ham_sparse*time, initial_state)

### Exercise

Fill in the code cell below to construct a circuit which applies $\exp(-i H t)$ using the method described above

In [27]:
# Initialize qubits
qubits = cirq.LineQubit.range(n_qubits)

# Write code below to create the circuit
# You should define the `circuit` variable here
# ---------------------------------------------


# ---------------------------------------------

# Apply the circuit to the initial state
result = circuit.apply_unitary_effect_to_state(initial_state)

# Compute the fidelity with the correct final state
fidelity = abs(np.dot(final_state, result.conj()))**2

# Print fidelity; it should be 1
print(fidelity)

0.08926042490120051


## Solutions to exercises

In [28]:
# Initialize qubits
qubits = cirq.LineQubit.range(n_qubits)

# Write code below to create the circuit
# You should define the `circuit` variable here
# ---------------------------------------------
def zz_interaction(i, j, a, b):
    return ofc.ZZGate(duration=ising_model.terms[(i, 'Z'), (j, 'Z')]).on(a, b)
circuit = cirq.Circuit.from_ops(ofc.swap_network(qubits, zz_interaction))
circuit.append(ofc.swap_network(qubits))
# ---------------------------------------------

# Apply the circuit to the initial state
result = circuit.apply_unitary_effect_to_state(initial_state)

# Compute the fidelity with the correct final state
fidelity = abs(np.dot(final_state, result.conj()))**2

# Print fidelity; it should be 1
print(fidelity)

0.008334497743611877


In [29]:
# Initialize qubits
qubits = cirq.LineQubit.range(n_qubits)

# Write code below to create the circuit
# You should define the `circuit` variable here
# ---------------------------------------------
def exponentiate_quad_ham(qubits, quad_ham):
    basis_change_matrix = quad_ham.diagonalizing_bogoliubov_transform()
    orbital_energies, _ = quad_ham.orbital_energies()
    
    yield cirq.inverse(
        ofc.bogoliubov_transform(qubits, basis_change_matrix))
    for i in range(len(qubits)):
        yield cirq.RotZGate(rads=-orbital_energies[i]).on(qubits[i])
    yield ofc.bogoliubov_transform(qubits, basis_change_matrix)

circuit = cirq.Circuit.from_ops(exponentiate_quad_ham(qubits, quad_ham))
# ---------------------------------------------

# Apply the circuit to the initial state
result = circuit.apply_unitary_effect_to_state(initial_state)

# Compute the fidelity with the correct final state
fidelity = abs(np.dot(final_state, result.conj()))**2

# Print fidelity; it should be 1
print(fidelity)

0.9999999999999938
